In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

from datetime import date
from pathlib import Path
import boto3

pd.set_option("display.max_columns", 150)

# Read Canary data in Pandas Array

In [2]:
data = pq.read_table('../data/CANARY/CANARY/PERSON_SUMMARY').to_pandas()

In [3]:
data.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB  \
0  2638452  M0006106300       37330  Medicare Advantage 175  Medicare   
1  2706146  M0010332600       36550  Medicare Advantage 125  Medicare   
2  4122088  CD116486701       35938          365 LifePoints        LG   
3  4254340  CD127136501       35938          365 LifePoints        LG   
4  4275621  CD128714401       35938          365 LifePoints       ASO   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
4  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   

   N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  \
0                      1    2020-01-26         2020-01-26   
1                      9    2020-05-27         2020-08-31   
2                     17    2019-11-20         2020-05-29   
3                     17    2019-02-04         2019-07-24   
4                     16    2020-01-24         2020-09-01   

    LAST_COMPLETED_MILESTONE  
0              REGISTER_DATE  
1   9TH_LESSON_COMPLETE_DATE  
2  17TH_LESSON_COMPLETE_DATE  
3  17TH_LESSON_COMPLETE_DATE  
4  16TH_LESSON_COMPLETE_DATE

In [4]:
data.describe()

N_MILESTONES_ACHIEVED
count            1237.000000
mean               12.046079
std                 6.852017
min                 1.000000
25%                 5.000000
50%                16.000000
75%                17.000000
max                25.000000

### All patients in Canary Data are in the Treatment Group

In [5]:

data['TREATED'] = 1


In [6]:
data.head()

PERS_ID       MBR_ID GROUPNUMBER               GROUPNAME       LOB  \
0  2638452  M0006106300       37330  Medicare Advantage 175  Medicare   
1  2706146  M0010332600       36550  Medicare Advantage 125  Medicare   
2  4122088  CD116486701       35938          365 LifePoints        LG   
3  4254340  CD127136501       35938          365 LifePoints        LG   
4  4275621  CD128714401       35938          365 LifePoints       ASO   

                                           FILE_NAME   FILE_DATE  \
0  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
1  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
2  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
3  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   
4  s3://cdphp-s3-us-e-p-datalake/RAW/NATIVE/WELLT...  2020-09-07   

   N_MILESTONES_ACHIEVED REGISTER_DATE LAST_ACTIVITY_DATE  \
0                      1    2020-01-26         2020-01-26   
1                      9    2020-05-27         2020-08-31   
2                     17    2019-11-20         2020-05-29   
3                     17    2019-02-04         2019-07-24   
4                     16    2020-01-24         2020-09-01   

    LAST_COMPLETED_MILESTONE  TREATED  
0              REGISTER_DATE        1  
1   9TH_LESSON_COMPLETE_DATE        1  
2  17TH_LESSON_COMPLETE_DATE        1  
3  17TH_LESSON_COMPLETE_DATE        1  
4  16TH_LESSON_COMPLETE_DATE        1

### Read Eligible PMPM data

In [7]:
eligible = pd.read_pickle('../temp/eligible.p')
eligible['MBR_GNDR'].replace(['F', 'M'], [0,1], inplace = True)

In [8]:
eligible.describe()

AGE_AT_MIDMONTH     ALLOW_AMT       CC_ADHD  CC_ALZHEIMER  \
count     7.666216e+06  7.666216e+06  7.666216e+06  7.666216e+06   
mean      5.064458e+01  8.486720e+02  3.471191e-02  6.833489e-03   
std       1.940056e+01  4.741002e+03  1.830492e-01  8.238199e-02   
min       0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%       3.600000e+01  0.000000e+00  0.000000e+00  0.000000e+00   
50%       5.300000e+01  1.420000e+02  0.000000e+00  0.000000e+00   
75%       6.500000e+01  4.940000e+02  0.000000e+00  0.000000e+00   
max       1.200000e+02  2.107412e+06  1.000000e+00  1.000000e+00   

       CC_ALZHEIMER_PLUS        CC_AMI     CC_ANEMIA  CC_ANXIETY_DISORDER  \
count       7.666216e+06  7.666216e+06  7.666216e+06         7.666216e+06   
mean        2.076970e-02  3.504597e-03  8.151179e-02         2.059938e-01   
std         1.426125e-01  5.909582e-02  2.736195e-01         4.044260e-01   
min         0.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
25%         0.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
50%         0.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
75%         0.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
max         1.000000e+00  1.000000e+00  1.000000e+00         1.000000e+00   

       CC_ARTHRITIS     CC_ASTHMA  CC_ATRIAL_FIB     CC_AUTISM    CC_BIPOLAR  \
count  7.666216e+06  7.666216e+06   7.666216e+06  7.666216e+06  7.666216e+06   
mean   1.595076e-01  6.143044e-02   3.691600e-02  5.047601e-03  3.612317e-02   
std    3.661488e-01  2.401182e-01   1.885556e-01  7.086694e-02  1.865966e-01   
min    0.000000e+00  0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
25%    0.000000e+00  0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
50%    0.000000e+00  0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
75%    0.000000e+00  0.000000e+00   0.000000e+00  0.000000e+00  0.000000e+00   
max    1.000000e+00  1.000000e+00   1.000000e+00  1.000000e+00  1.000000e+00   

             CC_BPH  CC_BREAST_CANCER   CC_CATARACT  CC_CEREBRAL_PALSY  \
count  7.666216e+06      7.666216e+06  7.666216e+06       7.666216e+06   
mean   3.128780e-02      1.880471e-02  7.718775e-02       1.956637e-03   
std    1.740944e-01      1.358348e-01  2.668891e-01       4.419059e-02   
min    0.000000e+00      0.000000e+00  0.000000e+00       0.000000e+00   
25%    0.000000e+00      0.000000e+00  0.000000e+00       0.000000e+00   
50%    0.000000e+00      0.000000e+00  0.000000e+00       0.000000e+00   
75%    0.000000e+00      0.000000e+00  0.000000e+00       0.000000e+00   
max    1.000000e+00      1.000000e+00  1.000000e+00       1.000000e+00   

       CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER       CC_COPD  \
count       7.666216e+06          7.666216e+06  7.666216e+06   
mean        1.073240e-01          4.493351e-03  4.785712e-02   
std         3.095248e-01          6.688170e-02  2.134639e-01   
min         0.000000e+00          0.000000e+00  0.000000e+00   
25%         0.000000e+00          0.000000e+00  0.000000e+00   
50%         0.000000e+00          0.000000e+00  0.000000e+00   
75%         0.000000e+00          0.000000e+00  0.000000e+00   
max         1.000000e+00          1.000000e+00  1.000000e+00   

       CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  \
count        7.666216e+06   7.666216e+06             7.666216e+06   
mean         2.959087e-03   2.033544e-01             6.668088e-03   
std          5.431695e-02   4.024940e-01             8.138566e-02   
min          0.000000e+00   0.000000e+00             0.000000e+00   
25%          0.000000e+00   0.000000e+00             0.000000e+00   
50%          0.000000e+00   0.000000e+00             0.000000e+00   
75%          0.000000e+00   0.000000e+00             0.000000e+00   
max          1.000000e+00   1.000000e+00             1.000000e+00   

       CC_DEVELOP_DELAYS   CC_DIABETES  CC_DISAB_DX_CNT     CC_DX_CNT  \
count       7.666216e+06  7.666216e+06  

In [9]:
eligible.MYR.max(), eligible.MYR.min()

(202009, 201701)

In [10]:
canary_ids = data.PERS_ID.unique()
eligible_ids = eligible.PERS_ID.unique()


### Pick Covariates for matching

In [11]:
non_cov = ['PERS_ID', 'MYR','ALLOW_AMT']
covariates = np.setdiff1d(list(eligible.columns), non_cov)

### Merge PMPM with Canary

In [12]:
df = eligible.merge(data,how = 'left', on = 'PERS_ID')

In [13]:
df.loc[df.TREATED.isnull(), 'TREATED'] = 0

In [14]:
df.TREATED.sum()

42002.0

In [15]:
del data
del eligible

In [22]:
df.PERS_ID = df.PERS_ID.astype(int)

In [23]:
filte = df.PERS_ID.isin([-1,-3])
df = df[~filte]

In [63]:
df.MYR.unique()

array([201812, 201709, 202002, 202006, 201804, 202007, 201705, 202008,
       201710, 201701, 201708, 201712, 201810, 201907, 201702, 201811,
       201707, 201808, 201908, 201801, 201905, 201906, 201911, 201807,
       201910, 201904, 201902, 202001, 201706, 201703, 201909, 201903,
       201704, 202003, 202004, 202005, 201912, 201711, 202009, 201806,
       201805, 201809, 201802, 201901, 201803])

### Setting all the covariates whatever is related to diagnostic codes plus age, gender

### Create Registartion and End Date for controls

In [68]:

#FUNCTION TO SET CRITERIA FOR REGISTRATION DATE
def add_dates(df):
    if not any(df.REGISTER_DATE.isnull()):
        idi = df.PERS_ID if len(df) <= 1 else df.PERS_ID.iloc[0]
        rd  = df.REGISTER_DATE if len(df) <= 1 else df.REGISTER_DATE.iloc[0]
        la  = df.LAST_ACTIVITY_DATE if len(df) <= 1 else df.LAST_ACTIVITY_DATE.iloc[0]
        return pd.DataFrame([[idi, rd, la]], columns = ['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE'])
    else:
        if len(df.MYR) >1:
            sortmy = df.MYR.sort_values()
            med = str(sortmy.iloc[int(len(df.MYR)/2)])
            mx  = str(df.MYR.max())
           # print('Here',int(med[4:6]), med)

        else:
            med = str(df.MYR.iloc[0])
            mx  = str(df.MYR.iloc[0])
           # print('Here',int(med[4:6]))
        rd = date(year = int(med[0:4]), month = int(med[4:6]), day = 1)
        la = date(year = int(mx[0:4]), month = int(mx[4:6]), day = 1)
       # print(df.PERS_ID)
        idi = df.PERS_ID.iloc[0]
       # rd  = df.REGISTER_DATE if len(df) <= 1 else df.REGISTER_DATE.iloc[0]
        #la  = df.LAST_ACTIVITY_DATE if len(df) <= 1 else df.LAST_ACTIVITY_DATE.iloc[0]
       # print(df.PERS_ID)
        #ghghgh
            
        return pd.DataFrame([[idi, rd, la]], columns = ['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE'])

In [74]:
treated = df.TREATED == 1
data_t  = df[treated].reset_index(drop = True)


In [107]:
data_c = df[~treated]
del df

In [108]:
data_c.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570719               94       5251        0             0   
1  2570719               93        127        0             0   
2  2570719               95         67        0             0   
3  2570719               95         20        0             0   
4  2570719               93        859        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             0   
2                  0       0          0                    0             1   
3                  0       0          0                    0             0   
4                  0       0          0                    0             0   

   CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  \
0          0              0          0           0       0                 0   
1          0              0          0           0       0                 0   
2          0              0          0           0       0                 0   
3          0              0          0           0       1                 0   
4          0              0          0           0       0                 0   

   CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  \
0            0                  0                  0                     1   
1            0                  0                  0                     0   
2            0                  0                  1                     1   
3            0                  0                  1                     0   
4            0                  0                  0                     0   

   CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  \
0        0                   0              0                        0   
1        0                   0              0                        0   
2        1                   0              0                        0   
3        1                   0              0                        0   
4        0                   0              0                        0   

   CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  \
0                  0            0                1          6   
1                  0            0                0          3   
2                  0            0                0         11   
3                  0            0                0          9   
4                  0            0                0          3   

   CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  \
0                      0            0                1            0   
1                      0            0                0            0   
2                      0            0                1            0   
3                      0            0                0            0   
4                      0            0                0            0   

   CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  \
0                  1                 1               0                     0   
1                  0                 0               0                     0   
2                  0                 1               0                     0   
3                  0                 1               0                     0   
4                  0                 1               0                     0   

   CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     0                       0   
3                       0                     0                       0   
4                       0                     0                       0   

In [137]:
data_c.drop(labels = ['REGISTER_DATE', 'LAST_ACTIVITY_DATE'], inplace = True, axis = 1)

In [138]:
data_c.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570719               94       5251        0             0   
1  2570719               93        127        0             0   
2  2570719               95         67        0             0   
3  2570719               95         20        0             0   
4  2570719               93        859        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             0   
2                  0       0          0                    0             1   
3                  0       0          0                    0             0   
4                  0       0          0                    0             0   

   CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  \
0          0              0          0           0       0                 0   
1          0              0          0           0       0                 0   
2          0              0          0           0       0                 0   
3          0              0          0           0       1                 0   
4          0              0          0           0       0                 0   

   CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  \
0            0                  0                  0                     1   
1            0                  0                  0                     0   
2            0                  0                  1                     1   
3            0                  0                  1                     0   
4            0                  0                  0                     0   

   CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  \
0        0                   0              0                        0   
1        0                   0              0                        0   
2        1                   0              0                        0   
3        1                   0              0                        0   
4        0                   0              0                        0   

   CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  \
0                  0            0                1          6   
1                  0            0                0          3   
2                  0            0                0         11   
3                  0            0                0          9   
4                  0            0                0          3   

   CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  \
0                      0            0                1            0   
1                      0            0                0            0   
2                      0            0                1            0   
3                      0            0                0            0   
4                      0            0                0            0   

   CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  \
0                  1                 1               0                     0   
1                  0                 0               0                     0   
2                  0                 1               0                     0   
3                  0                 1               0                     0   
4                  0                 1               0                     0   

   CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     0                       0   
3                       0                     0                       0   
4                       0                     0                       0   

In [130]:
big_list = []
def control_dates(group):
    dates_srt = group.MYR2
    rd      = dates_srt.iloc[int(len(dates_srt)/2)]
    lad     = dates_srt.iloc[-1]
    idi     = group.PERS_ID.iloc[0]
    
    big_list.append([idi, rd, lad])
    
   # return pd.DataFrame([[idi, lad, idi]], columns = ['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE'])
    


In [124]:
#FInd register dates
control_df = data_c[['PERS_ID',  'MYR']].copy()
control_df = control_df.sort_values(by = 'MYR').reset_index(drop = True)
control_df['MYR2'] = control_df.MYR.apply(lambda x: date(year = int(str(x)[0:4]), month = int(str(x)[4:6]), day = 1))

In [125]:
control_df.head()

PERS_ID     MYR        MYR2
0  4264241  201701  2017-01-01
1  3391715  201701  2017-01-01
2  4263003  201701  2017-01-01
3  3090759  201701  2017-01-01
4  3057467  201701  2017-01-01

In [131]:
control_df.groupby(by = 'PERS_ID').apply(control_dates)


Empty DataFrame
Columns: []
Index: []

In [133]:
control_df_final = pd.DataFrame(big_list, columns = ['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE'])

In [139]:
control_df_final.head()

PERS_ID REGISTER_DATE LAST_ACTIVITY_DATE
0  2561296    2019-07-01         2020-06-01
1  2562628    2017-08-01         2018-03-01
2  2562714    2017-08-01         2017-10-01
3  2562798    2018-12-01         2020-09-01
4  2563150    2017-11-01         2018-09-01

In [140]:
final_control = data_c.merge(control_df_final, how = 'left', on = 'PERS_ID')

In [142]:
final_control.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2570719               94       5251        0             0   
1  2570719               93        127        0             0   
2  2570719               95         67        0             0   
3  2570719               95         20        0             0   
4  2570719               93        859        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             0   
2                  0       0          0                    0             1   
3                  0       0          0                    0             0   
4                  0       0          0                    0             0   

   CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  \
0          0              0          0           0       0                 0   
1          0              0          0           0       0                 0   
2          0              0          0           0       0                 0   
3          0              0          0           0       1                 0   
4          0              0          0           0       0                 0   

   CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  \
0            0                  0                  0                     1   
1            0                  0                  0                     0   
2            0                  0                  1                     1   
3            0                  0                  1                     0   
4            0                  0                  0                     0   

   CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  \
0        0                   0              0                        0   
1        0                   0              0                        0   
2        1                   0              0                        0   
3        1                   0              0                        0   
4        0                   0              0                        0   

   CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  \
0                  0            0                1          6   
1                  0            0                0          3   
2                  0            0                0         11   
3                  0            0                0          9   
4                  0            0                0          3   

   CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  \
0                      0            0                1            0   
1                      0            0                0            0   
2                      0            0                1            0   
3                      0            0                0            0   
4                      0            0                0            0   

   CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  \
0                  1                 1               0                     0   
1                  0                 0               0                     0   
2                  0                 1               0                     0   
3                  0                 1               0                     0   
4                  0                 1               0                     0   

   CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     0                       0   
3                       0                     0                       0   
4                       0                     0                       0   

In [187]:
final_control[['PERS_ID', 'REGISTER_DATE', 'LAST_ACTIVITY_DATE']].to_pickle('../temp/controls.p')

In [153]:
cov_control = final_control[covariates]
cov_control['PERS_ID'] = final_control['PERS_ID']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [97]:
#function to get covariates
def get_cov(df):
    dates = df.MYR.apply(lambda x: date(year = int(str(x)[0:4]), month = int(str(x)[4:6]), day = 1))
    ind_min = np.argmin(np.abs(dates - df.REGISTER_DATE.iloc[0]))
    row = df.iloc[ind_min]
    return row

In [103]:
x = data_t.groupby(by = 'PERS_ID').apply(get_cov)
x.index = np.arange(len(x))

In [152]:
cov_tr = x[covariates]
cov_tr['PERS_ID'] = x.PERS_ID

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
cov_tr.head()

AGE_AT_MIDMONTH  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  \
0               81        0             0                  0       0   
1               82        0             0                  0       0   
2               78        0             0                  0       0   
3               78        0             0                  0       0   
4               77        0             0                  0       0   

   CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  \
0          0                    1             0          0              0   
1          0                    1             0          0              0   
2          0                    0             0          0              0   
3          0                    0             1          0              0   
4          0                    0             0          1              0   

   CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  \
0          0           0       0                 0            0   
1          0           0       0                 0            0   
2          0           0       1                 0            1   
3          0           1       0                 0            1   
4          0           0       0                 0            0   

   CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  \
0                  0                  0                     0        0   
1                  0                  0                     0        0   
2                  0                  0                     0        0   
3                  0                  0                     0        0   
4                  0                  0                     0        0   

   CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  \
0                   0              0                        0   
1                   0              0                        0   
2                   0              0                        0   
3                   0              1                        0   
4                   0              0                        0   

   CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  \
0                  0            0                1          1   
1                  0            0                0          3   
2                  0            1                0          7   
3                  0            0                0          8   
4                  0            0                1          5   

   CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  \
0                      0            0                0            0   
1                      0            0                0            0   
2                      0            0                0            0   
3                      0            0                0            0   
4                      0            0                0            1   

   CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  \
0                  1                 0               0                     0   
1                  0                 0               0                     0   
2                  0                 0               0                     0   
3                  0                 0               0                     0   
4                  1                 0               0                     0   

   CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     0                       0   
3                       0                     0                       0   
4                       0                     0                       0   

   CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  \
0                           0               0        

## From here we need to run with the same random seed 

In [ ]:
np.random.seed(seed=0)

In [154]:
cov_tr['TREATED'] = 1
cov_control['TREATED'] = 0

data_filt2 = pd.concat((cov_tr, cov_control), axis = 0, ignore_index=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [155]:
data_filt2.shape, cov_tr.shape, cov_control.shape

((7625407, 73), (1226, 73), (7624181, 73))

## Propensity Score Matching

In [156]:
covariates_v2 = covariates.copy()

In [157]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X, Y = data_filt2[covariates_v2], data_filt2.TREATED

ss = StandardScaler()
pca = PCA(n_components=5)
Xtr   = pca.fit_transform(ss.fit_transform(X.values))

/usr/local/lib64/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib64/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [158]:
treated = data_filt2.TREATED == 1

In [159]:
data_filt2.shape, Xtr.shape

((7625407, 73), (7625407, 5))

In [160]:
data_treat = data_filt2[treated].reset_index(drop = True)
data_contr= data_filt2[~treated].reset_index(drop = True)

In [161]:
data_treat.shape, data_contr.shape

((1226, 73), (7624181, 73))

In [167]:
lg = LogisticRegression(fit_intercept=True)
lg = lg.fit(Xtr, Y.values)

/usr/local/lib64/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [168]:
probas_t = lg.predict_proba(Xtr[treated])[:,1]
probas_c = lg.predict_proba(Xtr[~treated])[:,1]

In [169]:
probas_t.shape, probas_c.shape

((1226,), (7624181,))

In [170]:
print("min-max probabilities of treatment and control after lg fit")
probas_c.max(), probas_c.min(), probas_t.max(), probas_t.min()

min-max probabilities of treatment and control after lg fit


(0.0018232301553173908,
 1.5104705138979356e-23,
 0.000569447577373133,
 9.207646195158471e-07)

### Match Control and Treatment Units

In [178]:
np.arange(len(probas_c)).max()

7624180

In [177]:
from sklearn.metrics.pairwise import euclidean_distances

control_matches = []
dist_min =[]
for i in np.arange(len(probas_t)):
    if i%50 == 0:
        print(i)
    dist = np.sqrt(euclidean_distances(X=probas_t[i].reshape([-1,1]), Y=probas_c[:,np.newaxis]))
    dist_min.append(dist.min())
    control_matches.append(np.argmin(dist))

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


IndexError: index 1226 is out of bounds for axis 0 with size 1226

In [180]:
data_c

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0        2570719               94       5251        0             0   
1        2570719               93        127        0             0   
2        2570719               95         67        0             0   
3        2570719               95         20        0             0   
4        2570719               93        859        0             0   
...          ...              ...        ...      ...           ...   
7666211  5041693               79        521        1             0   
7666212  5041693               79        821        0             0   
7666213  5041693               80        197        1             0   
7666214  5047021               11         14        0             0   
7666215  5047384               53          0        0             0   

         CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  \
0                        0       0          0                    0   
1                        0       0          0                    0   
2                        0       0          0                    0   
3                        0       0          0                    0   
4                        0       0          0                    0   
...                    ...     ...        ...                  ...   
7666211                  0       0          1                    0   
7666212                  0       0          1                    0   
7666213                  0       0          1                    0   
7666214                  0       0          0                    0   
7666215                  0       0          0                    0   

         CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  \
0                   1          0              0          0           0   
1                   0          0              0          0           0   
2                   1          0              0          0           0   
3                   0          0              0          0           0   
4                   0          0              0          0           0   
...               ...        ...            ...        ...         ...   
7666211             0          0              0          0           0   
7666212             0          0              0          0           0   
7666213             0          0              0          0           0   
7666214             0          0              0          0           0   
7666215             0          0              0          0           0   

         CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  \
0             0                 0            0                  0   
1             0                 0            0                  0   
2             0                 0            0                  0   
3             1                 0            0                  0   
4             0                 0            0                  0   
...         ...               ...          ...                ...   
7666211       0                 0            0                  0   
7666212       0                 0            0                  0   
7666213       0                 0            0                  0   
7666214       0                 0            0                  0   
7666215       0                 0            0                  0   

         CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  \
0                        0                     1        0                   0   
1                        0                     0        0                   0   
2                        1                     1        1                   0   
3                        1                     0        1                   0   
4                        0                     0        0                   0   
...                    ...                   ...      ...                 ...   
7666211                  0    

In [182]:

data_cc = data_c.iloc[control_matches]

In [183]:
data_cc.shape

(1226, 83)

In [ ]:
data_treat.shape, data_t.shape, data_cc.shape, data_c.shape

### Caclculate and Save the Treated and Matched Control IDs

In [184]:
matched_controls_ID = data_cc.PERS_ID.unique()
treated_ids         = cov_tr.PERS_ID.unique()
ids_to_match = {'treated': treated_ids, 'control':matched_controls_ID}

In [185]:
len(matched_controls_ID), len(treated_ids)

(957, 1226)

In [186]:
import pickle as pkl
pkl.dump(ids_to_match, open('../temp/matching_ids.p', 'wb') )

In [ ]:
dist_med